<a href="https://colab.research.google.com/github/YUTA-NIKI/lecture_deeplearning_matsuo/blob/master/chap06_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/lecture_master/2019_summer/deeplearning_matsuo/data/chap06"
data_path = "/content/drive/My Drive/lecture_master/2019_summer/deeplearning_matsuo/data/chap06"

t_train.npy  x_test.npy  x_train.npy


In [0]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

try:
    del [
        tf.app,
        tf.compat,
        tf.contrib,
        tf.estimator,
        tf.gfile,
        tf.graph_util,
        tf.image,
        tf.initializers,
        tf.keras,
        tf.layers,
        tf.logging,
        tf.losses,
        tf.metrics,
        tf.python_io,
        tf.resource_loader,
        tf.saved_model,
        tf.sets,
        tf.summary,
        tf.sysconfig,
        tf.test
    ]
    
except AttributeError:
    print('Unrequired modules are already deleted (Skipped).')

def load_mnist():

    # 学習データ
    x_train = np.load(os.path.join(data_path,'x_train.npy'))
    t_train = np.load(os.path.join(data_path,'t_train.npy'))
    
    # テストデータ
    x_test = np.load(os.path.join(data_path,'x_test.npy'))

    x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
    x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255
    t_train = np.eye(10)[t_train.astype('int32').flatten()]

    return (x_train, x_test, t_train)

Optimizer: SGD

In [0]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

rng = np.random.RandomState(1234)
random_state = 42

### レイヤー定義 ###

class Conv:
    def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='VALID'):
        
        fan_in  = np.prod(filter_shape[:3])
        fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
        
        self.W = tf.Variable(rng.uniform(
            low=-np.sqrt(6/fan_in),
            high=np.sqrt(6/fan_in),
            size=filter_shape
        ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b')
        self.function = function
        self.strides = strides
        self.padding = padding
        
    def __call__(self, x):
        u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding) + self.b
        return self.function(u)
    
    
class Pooling:
    def __init__(self, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID'):
        self.ksize = ksize
        self.strides = strides
        self.padding = padding
        
    def __call__(self, x):
        return tf.nn.max_pool(x, ksize=self.ksize, strides=self.strides, padding=self.padding)
    
    
class Flatten:
    def __call__(self, x):
        return tf.reshape(x, (-1, np.prod(x.get_shape().as_list()[1:])))
    
    
class Dense:
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        
        self.W = tf.Variable(rng.uniform(
            low=-np.sqrt(6/in_dim),
            high=np.sqrt(6/in_dim),
            size=(in_dim, out_dim)
        ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros([out_dim]).astype('float32'))
        self.function = function
        
    def __call__(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)
    
    
def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))
    
### ネットワーク ###

x_train, x_test, t_train = load_mnist()
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train, test_size=0.1, random_state=random_state)

tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 28, 28, 1])
t = tf.placeholder(tf.float32, [None, 10])

h = Conv((5, 5, 1, 20), tf.nn.relu)(x)
h = Pooling((1, 2, 2, 1))(h)
h = Conv((5, 5, 20, 50), tf.nn.relu)(h)
h = Pooling((1, 2, 2, 1))(h)
h = Flatten()(h)
y = Dense(4*4*50, 10, tf.nn.softmax)(h)

cost = - tf.reduce_mean(tf.reduce_sum(t * tf_log(y), axis=1))
train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

### 学習 ###

n_epochs = 10
batch_size = 100
n_batches = x_train.shape[0]//batch_size


sess = tf.Session()
sess.run(tf.global_variables_initializer())


for epoch in range(n_epochs):
    x_train, t_train = shuffle(x_train, t_train)
    for batch in range(n_batches):
        start = batch * batch_size
        end = start + batch_size
        sess.run(train, feed_dict={x: x_train[start:end], t: t_train[start:end]})
            
    y_pred, cost_valid = sess.run([y, cost], feed_dict={x: x_valid, t: t_valid})
        
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
        epoch,
        cost_valid,
        accuracy_score(t_valid.argmax(axis=1), y_pred.argmax(axis=1))
    ))
# y_pred = sess.run(y, feed_dict={x: x_test})
# submission = pd.Series(y_pred.argmax(axis=1), name='label')
# submission.to_csv('/content/drive/My Drive/lecture_master/2019_summer/deeplearning_matsuo/homework/chap06/submission_pred.csv', header=True, index_label='id')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
EPOCH: 0, Valid Cost: 0.588, Valid Accuracy: 0.783
EPOCH: 1, Valid Cost: 0.501, Valid Accuracy: 0.820
EPOCH: 2, Valid Cost: 0.467, Valid Accuracy: 0.830
EPOCH: 3, Valid Cost: 0.440, Valid Accuracy: 0.845
EPOCH: 4, Valid Cost: 0.426, Valid Accuracy: 0.852
EPOCH: 5, Valid Cost: 0.402, Valid Accuracy: 0.857
EPOCH: 6, Valid Cost: 0.393, Valid Accuracy: 0.860
EPOCH: 7, Valid Cost: 0.390, Valid Accuracy: 0.861
EPOCH: 8, Valid Cost: 0.374, Valid Accuracy: 0.865
EPOCH: 9, Valid Cost: 0.372, Valid Accuracy: 0.865


Optimizer: Adam

In [0]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

rng = np.random.RandomState(1234)
random_state = 42

### レイヤー定義 ###

class Conv:
    def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='VALID'):
        
        fan_in  = np.prod(filter_shape[:3])
        fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
        
        self.W = tf.Variable(rng.uniform(
            low=-np.sqrt(6/fan_in),
            high=np.sqrt(6/fan_in),
            size=filter_shape
        ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b')
        self.function = function
        self.strides = strides
        self.padding = padding
        
    def __call__(self, x):
        u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding) + self.b
        return self.function(u)
    
    
class Pooling:
    def __init__(self, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID'):
        self.ksize = ksize
        self.strides = strides
        self.padding = padding
        
    def __call__(self, x):
        return tf.nn.max_pool(x, ksize=self.ksize, strides=self.strides, padding=self.padding)
    
    
class Flatten:
    def __call__(self, x):
        return tf.reshape(x, (-1, np.prod(x.get_shape().as_list()[1:])))
    
    
class Dense:
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        
        self.W = tf.Variable(rng.uniform(
            low=-np.sqrt(6/in_dim),
            high=np.sqrt(6/in_dim),
            size=(in_dim, out_dim)
        ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros([out_dim]).astype('float32'))
        self.function = function
        
    def __call__(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)
    
    
def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))
    
### ネットワーク ###

x_train, x_test, t_train = load_mnist()
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train, test_size=0.1, random_state=random_state)

tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 28, 28, 1])
t = tf.placeholder(tf.float32, [None, 10])

h = Conv((5, 5, 1, 20), tf.nn.relu)(x)
h = Pooling((1, 2, 2, 1))(h)
h = Conv((5, 5, 20, 50), tf.nn.relu)(h)
h = Pooling((1, 2, 2, 1))(h)
h = Flatten()(h)
h = Dense(4*4*50, 200, tf.nn.relu)(h)
h = Dense(200, 200, tf.nn.relu)(h)
y = Dense(200, 10, tf.nn.softmax)(h)

cost = - tf.reduce_mean(tf.reduce_sum(t * tf_log(y), axis=1))
train = tf.train.AdamOptimizer(0.001).minimize(cost)

### 学習 ###

n_epochs = 20
batch_size = 100
n_batches = x_train.shape[0]//batch_size


sess = tf.Session()
sess.run(tf.global_variables_initializer())


for epoch in range(n_epochs):
    x_train, t_train = shuffle(x_train, t_train)
    for batch in range(n_batches):
        start = batch * batch_size
        end = start + batch_size
        sess.run(train, feed_dict={x: x_train[start:end], t: t_train[start:end]})
            
    y_pred, cost_valid = sess.run([y, cost], feed_dict={x: x_valid, t: t_valid})
        
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
        epoch,
        cost_valid,
        accuracy_score(t_valid.argmax(axis=1), y_pred.argmax(axis=1))
    ))
    
# y_pred = sess.run(y, feed_dict={x: x_test})
# submission = pd.Series(y_pred.argmax(axis=1), name='label')
# submission.to_csv('/content/drive/My Drive/lecture_master/2019_summer/deeplearning_matsuo/homework/chap06/submission_pred.csv', header=True, index_label='id')

EPOCH: 0, Valid Cost: 0.363, Valid Accuracy: 0.864
EPOCH: 1, Valid Cost: 0.306, Valid Accuracy: 0.891
EPOCH: 2, Valid Cost: 0.315, Valid Accuracy: 0.888
EPOCH: 3, Valid Cost: 0.281, Valid Accuracy: 0.897
EPOCH: 4, Valid Cost: 0.255, Valid Accuracy: 0.906
EPOCH: 5, Valid Cost: 0.281, Valid Accuracy: 0.899
EPOCH: 6, Valid Cost: 0.265, Valid Accuracy: 0.908
EPOCH: 7, Valid Cost: 0.267, Valid Accuracy: 0.912
EPOCH: 8, Valid Cost: 0.295, Valid Accuracy: 0.907
EPOCH: 9, Valid Cost: 0.309, Valid Accuracy: 0.902
EPOCH: 10, Valid Cost: 0.319, Valid Accuracy: 0.901
EPOCH: 11, Valid Cost: 0.325, Valid Accuracy: 0.910
EPOCH: 12, Valid Cost: 0.355, Valid Accuracy: 0.905
EPOCH: 13, Valid Cost: 0.384, Valid Accuracy: 0.906
EPOCH: 14, Valid Cost: 0.366, Valid Accuracy: 0.903
EPOCH: 15, Valid Cost: 0.367, Valid Accuracy: 0.909
EPOCH: 16, Valid Cost: 0.422, Valid Accuracy: 0.897
EPOCH: 17, Valid Cost: 0.446, Valid Accuracy: 0.908
EPOCH: 18, Valid Cost: 0.480, Valid Accuracy: 0.899
EPOCH: 19, Valid Cost:

Optimizer: Adam<br>
with Dropout

In [0]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

rng = np.random.RandomState(1234)
random_state = 42

### レイヤー定義 ###

class Conv:
    def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='VALID'):
        
        fan_in  = np.prod(filter_shape[:3])
        fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
        
        self.W = tf.Variable(rng.uniform(
            low=-np.sqrt(6/fan_in),
            high=np.sqrt(6/fan_in),
            size=filter_shape
        ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b')
        self.function = function
        self.strides = strides
        self.padding = padding
        
    def __call__(self, x):
        u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding) + self.b
        return self.function(u)
    
    
class Pooling:
    def __init__(self, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID'):
        self.ksize = ksize
        self.strides = strides
        self.padding = padding
        
    def __call__(self, x):
        return tf.nn.max_pool(x, ksize=self.ksize, strides=self.strides, padding=self.padding)
    
    
class Flatten:
    def __call__(self, x):
        return tf.reshape(x, (-1, np.prod(x.get_shape().as_list()[1:])))
    
    
class Dense:
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        
        self.W = tf.Variable(rng.uniform(
            low=-np.sqrt(6/in_dim),
            high=np.sqrt(6/in_dim),
            size=(in_dim, out_dim)
        ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros([out_dim]).astype('float32'))
        self.function = function
        
    def __call__(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)
    
class Dropout:
    def __init__(self, dropout_rate=0.0):
        self.dropout_rate = dropout_rate
        
    def __call__(self, x):
        return tf.cond(
            pred=is_training,
            true_fn=lambda: tf.nn.dropout(x, rate=self.dropout_rate),
            false_fn=lambda: x
        )
    
def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))
    
### ネットワーク ###

x_train, x_test, t_train = load_mnist()
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train, test_size=0.1, random_state=random_state)

tf.reset_default_graph()
tf.set_random_seed(34)

x = tf.placeholder(tf.float32, [None, 28, 28, 1])
t = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

h = Conv((5, 5, 1, 20), tf.nn.relu)(x)
h = Pooling((1, 2, 2, 1))(h)
h = Conv((5, 5, 20, 50), tf.nn.relu)(h)
h = Pooling((1, 2, 2, 1))(h)
h = Flatten()(h)
h = Dense(4*4*50, 200, tf.nn.relu)(h)
h = Dropout(dropout_rate=0.5)(h)
h = Dense(200, 200, tf.nn.relu)(h)
h = Dropout(dropout_rate=0.5)(h)
y = Dense(200, 10, tf.nn.softmax)(h)

cost = - tf.reduce_mean(tf.reduce_sum(t * tf_log(y), axis=1))
train = tf.train.AdamOptimizer(0.001).minimize(cost)

### 学習 ###

n_epochs = 20
batch_size = 100
n_batches = x_train.shape[0]//batch_size


sess = tf.Session()
sess.run(tf.global_variables_initializer())


for epoch in range(n_epochs):
    x_train, t_train = shuffle(x_train, t_train, random_state=random_state)
    for batch in range(n_batches):
        start = batch * batch_size
        end = start + batch_size
        sess.run(train, feed_dict={x: x_train[start:end], t: t_train[start:end], is_training: True})
            
    y_pred, cost_valid = sess.run([y, cost], feed_dict={x: x_valid, t: t_valid, is_training: False})
        
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
        epoch,
        cost_valid,
        accuracy_score(t_valid.argmax(axis=1), y_pred.argmax(axis=1))
    ))
    
# y_pred = sess.run(y, feed_dict={x: x_test, is_training: False})
# submission = pd.Series(y_pred.argmax(axis=1), name='label')
# submission.to_csv('/content/drive/My Drive/lecture_master/2019_summer/deeplearning_matsuo/homework/chap06/submission_pred.csv', header=True, index_label='id')

EPOCH: 0, Valid Cost: 0.412, Valid Accuracy: 0.849
EPOCH: 1, Valid Cost: 0.335, Valid Accuracy: 0.875
EPOCH: 2, Valid Cost: 0.309, Valid Accuracy: 0.884
EPOCH: 3, Valid Cost: 0.288, Valid Accuracy: 0.892
EPOCH: 4, Valid Cost: 0.273, Valid Accuracy: 0.895
EPOCH: 5, Valid Cost: 0.261, Valid Accuracy: 0.901
EPOCH: 6, Valid Cost: 0.267, Valid Accuracy: 0.902
EPOCH: 7, Valid Cost: 0.252, Valid Accuracy: 0.905
EPOCH: 8, Valid Cost: 0.257, Valid Accuracy: 0.908
EPOCH: 9, Valid Cost: 0.259, Valid Accuracy: 0.908
EPOCH: 10, Valid Cost: 0.258, Valid Accuracy: 0.907
EPOCH: 11, Valid Cost: 0.267, Valid Accuracy: 0.904
EPOCH: 12, Valid Cost: 0.250, Valid Accuracy: 0.910
EPOCH: 13, Valid Cost: 0.260, Valid Accuracy: 0.906
EPOCH: 14, Valid Cost: 0.274, Valid Accuracy: 0.905
EPOCH: 15, Valid Cost: 0.271, Valid Accuracy: 0.906
EPOCH: 16, Valid Cost: 0.274, Valid Accuracy: 0.908
EPOCH: 17, Valid Cost: 0.269, Valid Accuracy: 0.907
EPOCH: 18, Valid Cost: 0.281, Valid Accuracy: 0.910
EPOCH: 19, Valid Cost:

Optimizer: Adam<br>
with l2 normalization<br>
with Dropout

In [0]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

rng = np.random.RandomState(1234)
random_state = 42

### レイヤー定義 ###

class Conv:
    def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='VALID'):
        
        fan_in  = np.prod(filter_shape[:3])
        fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
        
        self.W = tf.Variable(rng.uniform(
            low=-np.sqrt(6/fan_in),
            high=np.sqrt(6/fan_in),
            size=filter_shape
        ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b')
        self.function = function
        self.strides = strides
        self.padding = padding
        
        self.params = [self.W, self.b]
        
    def __call__(self, x):
        u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding) + self.b
        return self.function(u)
    
    
class Pooling:
    def __init__(self, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID'):
        self.ksize = ksize
        self.strides = strides
        self.padding = padding
        
        self.params = []
        
    def __call__(self, x):
        return tf.nn.max_pool(x, ksize=self.ksize, strides=self.strides, padding=self.padding)
    
    
class Flatten:
    def __init__(self):
        self.params = []
        
    def __call__(self, x):
        return tf.reshape(x, (-1, np.prod(x.get_shape().as_list()[1:])))
    
    
class Dense:
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        
        self.W = tf.Variable(rng.uniform(
            low=-np.sqrt(6/in_dim),
            high=np.sqrt(6/in_dim),
            size=(in_dim, out_dim)
        ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros([out_dim]).astype('float32'))
        self.function = function
        
        self.params = [self.W, self.b]
        
    def __call__(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)
    
class Dropout:
    def __init__(self, dropout_rate=0.0):
        self.dropout_rate = dropout_rate
        self.params = []
        
    def __call__(self, x):
        return tf.cond(
            pred=is_training,
            true_fn=lambda: tf.nn.dropout(x, rate=self.dropout_rate),
            false_fn=lambda: x
        )
    
def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))
    
def fprops(layers, h):
    for layer in layers:
        h = layer(h)
    return h

def get_params(layers):
    params_all = []
    for layer in layers:
        params = layer.params
        params_all.extend(params)
    return params_all

def compute_l2_reg(params):
    l2_reg = 0
    for param in params:
        l2_reg += tf.reduce_sum(tf.square(param))
    return l2_reg

### ネットワーク ###

x_train, x_test, t_train = load_mnist()
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train, test_size=0.1, random_state=random_state)

tf.reset_default_graph()
tf.set_random_seed(34)

x = tf.placeholder(tf.float32, [None, 28, 28, 1])
t = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

layers = [
    Conv((5, 5, 1, 20), tf.nn.relu),
    Pooling((1, 2, 2, 1)),
    Conv((5, 5, 20, 50), tf.nn.relu),
    Pooling((1, 2, 2, 1)),
    Flatten(),
    Dense(4*4*50, 200, tf.nn.relu),
    Dropout(dropout_rate=0.5),
    Dense(200, 200, tf.nn.relu),
    Dropout(dropout_rate=0.5),
    Dense(200, 10, tf.nn.softmax)
]

lmd = 1e-4 # l2正則化係数

y = fprops(layers, x)
params_all = get_params(layers)
l2_reg = compute_l2_reg(params_all)

cost = - tf.reduce_mean(tf.reduce_sum(t * tf_log(y), axis=1)) + lmd * l2_reg
train = tf.train.AdamOptimizer(0.001).minimize(cost)

### 学習 ###

n_epochs = 30
batch_size = 100
n_batches = x_train.shape[0]//batch_size


sess = tf.Session()
sess.run(tf.global_variables_initializer())


for epoch in range(n_epochs):
    x_train, t_train = shuffle(x_train, t_train, random_state=random_state)
    for batch in range(n_batches):
        start = batch * batch_size
        end = start + batch_size
        sess.run(train, feed_dict={x: x_train[start:end], t: t_train[start:end], is_training: True})
            
    y_pred, cost_valid = sess.run([y, cost], feed_dict={x: x_valid, t: t_valid, is_training: False})
        
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
        epoch,
        cost_valid,
        accuracy_score(t_valid.argmax(axis=1), y_pred.argmax(axis=1))
    ))
    
# y_pred = sess.run(y, feed_dict={x: x_test, is_training: False})
# submission = pd.Series(y_pred.argmax(axis=1), name='label')
# submission.to_csv('/content/drive/My Drive/lecture_master/2019_summer/deeplearning_matsuo/homework/chap06/submission_pred.csv', header=True, index_label='id')

EPOCH: 0, Valid Cost: 0.491, Valid Accuracy: 0.848
EPOCH: 1, Valid Cost: 0.411, Valid Accuracy: 0.875
EPOCH: 2, Valid Cost: 0.385, Valid Accuracy: 0.884
EPOCH: 3, Valid Cost: 0.360, Valid Accuracy: 0.893
EPOCH: 4, Valid Cost: 0.352, Valid Accuracy: 0.898
EPOCH: 5, Valid Cost: 0.345, Valid Accuracy: 0.896
EPOCH: 6, Valid Cost: 0.333, Valid Accuracy: 0.904
EPOCH: 7, Valid Cost: 0.324, Valid Accuracy: 0.906
EPOCH: 8, Valid Cost: 0.326, Valid Accuracy: 0.904
EPOCH: 9, Valid Cost: 0.336, Valid Accuracy: 0.905
EPOCH: 10, Valid Cost: 0.322, Valid Accuracy: 0.908
EPOCH: 11, Valid Cost: 0.326, Valid Accuracy: 0.910
EPOCH: 12, Valid Cost: 0.330, Valid Accuracy: 0.911
EPOCH: 13, Valid Cost: 0.331, Valid Accuracy: 0.909
EPOCH: 14, Valid Cost: 0.326, Valid Accuracy: 0.912
EPOCH: 15, Valid Cost: 0.340, Valid Accuracy: 0.907
EPOCH: 16, Valid Cost: 0.347, Valid Accuracy: 0.907
EPOCH: 17, Valid Cost: 0.333, Valid Accuracy: 0.912
EPOCH: 18, Valid Cost: 0.334, Valid Accuracy: 0.914
EPOCH: 19, Valid Cost:

Optimizer: Adam<br>
with Dropout<br>
without Pooling

In [0]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

rng = np.random.RandomState(1234)
random_state = 42

### レイヤー定義 ###

class Conv:
    def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='VALID'):
        
        fan_in  = np.prod(filter_shape[:3])
        fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
        
        self.W = tf.Variable(rng.uniform(
            low=-np.sqrt(6/fan_in),
            high=np.sqrt(6/fan_in),
            size=filter_shape
        ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b')
        self.function = function
        self.strides = strides
        self.padding = padding
        
        self.params = [self.W, self.b]
        
    def __call__(self, x):
        u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding) + self.b
        return self.function(u)
    
    
class Pooling:
    def __init__(self, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID'):
        self.ksize = ksize
        self.strides = strides
        self.padding = padding
        
        self.params = []
        
    def __call__(self, x):
        return tf.nn.max_pool(x, ksize=self.ksize, strides=self.strides, padding=self.padding)
    
    
class Flatten:
    def __init__(self):
        self.params = []
        
    def __call__(self, x):
        return tf.reshape(x, (-1, np.prod(x.get_shape().as_list()[1:])))
    
    
class Dense:
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        
        self.W = tf.Variable(rng.uniform(
            low=-np.sqrt(6/in_dim),
            high=np.sqrt(6/in_dim),
            size=(in_dim, out_dim)
        ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros([out_dim]).astype('float32'))
        self.function = function
        
        self.params = [self.W, self.b]
        
    def __call__(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)
    
class Dropout:
    def __init__(self, dropout_rate=0.0):
        self.dropout_rate = dropout_rate
        self.params = []
        
    def __call__(self, x):
        return tf.cond(
            pred=is_training,
            true_fn=lambda: tf.nn.dropout(x, rate=self.dropout_rate),
            false_fn=lambda: x
        )
    
def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))
    
def fprops(layers, h):
    for layer in layers:
        h = layer(h)
    return h

def get_params(layers):
    params_all = []
    for layer in layers:
        params = layer.params
        params_all.extend(params)
    return params_all

def compute_l2_reg(params):
    l2_reg = 0
    for param in params:
        l2_reg += tf.reduce_sum(tf.square(param))
    return l2_reg

### ネットワーク ###

x_train, x_test, t_train = load_mnist()
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train, test_size=0.1, random_state=random_state)

tf.reset_default_graph()
tf.set_random_seed(34)

x = tf.placeholder(tf.float32, [None, 28, 28, 1])
t = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

layers = [
    Conv((5, 5, 1, 20), tf.nn.relu),
    # Pooling((1, 2, 2, 1)),
    Conv((5, 5, 20, 50), tf.nn.relu),
    # Pooling((1, 2, 2, 1)),
    Flatten(),
    Dense(20*20*50, 200, tf.nn.relu),
    Dropout(dropout_rate=0.5),
    Dense(200, 200, tf.nn.relu),
    Dropout(dropout_rate=0.5),
    Dense(200, 10, tf.nn.softmax)
]

lmd = 1e-3 # l2正則化係数

y = fprops(layers, x)
params_all = get_params(layers)
l2_reg = compute_l2_reg(params_all)

cost = - tf.reduce_mean(tf.reduce_sum(t * tf_log(y), axis=1))
train = tf.train.AdamOptimizer(0.001).minimize(cost)

### 学習 ###

n_epochs = 30
batch_size = 100
n_batches = x_train.shape[0]//batch_size


sess = tf.Session()
sess.run(tf.global_variables_initializer())


for epoch in range(n_epochs):
    x_train, t_train = shuffle(x_train, t_train, random_state=random_state)
    for batch in range(n_batches):
        start = batch * batch_size
        end = start + batch_size
        sess.run(train, feed_dict={x: x_train[start:end], t: t_train[start:end], is_training: True})
            
    y_pred, cost_valid = sess.run([y, cost], feed_dict={x: x_valid, t: t_valid, is_training: False})
        
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
        epoch,
        cost_valid,
        accuracy_score(t_valid.argmax(axis=1), y_pred.argmax(axis=1))
    ))
    
# y_pred = sess.run(y, feed_dict={x: x_test, is_training: False})
# submission = pd.Series(y_pred.argmax(axis=1), name='label')
# submission.to_csv('/content/drive/My Drive/lecture_master/2019_summer/deeplearning_matsuo/homework/chap06/submission_pred.csv', header=True, index_label='id')

EPOCH: 0, Valid Cost: 0.368, Valid Accuracy: 0.869
EPOCH: 1, Valid Cost: 0.320, Valid Accuracy: 0.884
EPOCH: 2, Valid Cost: 0.285, Valid Accuracy: 0.894
EPOCH: 3, Valid Cost: 0.276, Valid Accuracy: 0.898
EPOCH: 4, Valid Cost: 0.260, Valid Accuracy: 0.906
EPOCH: 5, Valid Cost: 0.262, Valid Accuracy: 0.907
EPOCH: 6, Valid Cost: 0.257, Valid Accuracy: 0.908
EPOCH: 7, Valid Cost: 0.263, Valid Accuracy: 0.910
EPOCH: 8, Valid Cost: 0.279, Valid Accuracy: 0.911
EPOCH: 9, Valid Cost: 0.272, Valid Accuracy: 0.911
EPOCH: 10, Valid Cost: 0.272, Valid Accuracy: 0.913
EPOCH: 11, Valid Cost: 0.289, Valid Accuracy: 0.913
EPOCH: 12, Valid Cost: 0.292, Valid Accuracy: 0.909
EPOCH: 13, Valid Cost: 0.322, Valid Accuracy: 0.906
EPOCH: 14, Valid Cost: 0.323, Valid Accuracy: 0.913
EPOCH: 15, Valid Cost: 0.321, Valid Accuracy: 0.910
EPOCH: 16, Valid Cost: 0.383, Valid Accuracy: 0.909
EPOCH: 17, Valid Cost: 0.372, Valid Accuracy: 0.910
EPOCH: 18, Valid Cost: 0.377, Valid Accuracy: 0.912
EPOCH: 19, Valid Cost:

In [0]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

rng = np.random.RandomState(1234)
random_state = 42

### レイヤー定義 ###

class Conv:
    def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='VALID'):
        
        fan_in  = np.prod(filter_shape[:3])
        fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
        
        self.W = tf.Variable(rng.uniform(
            low=-np.sqrt(6/fan_in),
            high=np.sqrt(6/fan_in),
            size=filter_shape
        ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b')
        self.function = function
        self.strides = strides
        self.padding = padding
        
        self.params = [self.W, self.b]
        
    def __call__(self, x):
        u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding) + self.b
        return self.function(u)
    
    
class Pooling:
    def __init__(self, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID'):
        self.ksize = ksize
        self.strides = strides
        self.padding = padding
        
        self.params = []
        
    def __call__(self, x):
        return tf.nn.max_pool(x, ksize=self.ksize, strides=self.strides, padding=self.padding)
    
    
class Flatten:
    def __init__(self):
        self.params = []
        
    def __call__(self, x):
        return tf.reshape(x, (-1, np.prod(x.get_shape().as_list()[1:])))
    
    
class Dense:
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        
        self.W = tf.Variable(rng.uniform(
            low=-np.sqrt(6/in_dim),
            high=np.sqrt(6/in_dim),
            size=(in_dim, out_dim)
        ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros([out_dim]).astype('float32'))
        self.function = function
        
        self.params = [self.W, self.b]
        
    def __call__(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)
    
class Dropout:
    def __init__(self, dropout_rate=0.0):
        self.dropout_rate = dropout_rate
        self.params = []
        
    def __call__(self, x):
        return tf.cond(
            pred=is_training,
            true_fn=lambda: tf.nn.dropout(x, rate=self.dropout_rate),
            false_fn=lambda: x
        )
    
def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))
    
def fprops(layers, h):
    for layer in layers:
        h = layer(h)
    return h

def get_params(layers):
    params_all = []
    for layer in layers:
        params = layer.params
        params_all.extend(params)
    return params_all

def compute_l2_reg(params):
    l2_reg = 0
    for param in params:
        l2_reg += tf.reduce_sum(tf.square(param))
    return l2_reg

### ネットワーク ###

x_train, x_test, t_train = load_mnist()
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train, test_size=0.1, random_state=random_state)

tf.reset_default_graph()
tf.set_random_seed(34)

x = tf.placeholder(tf.float32, [None, 28, 28, 1])
t = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

layers = [
    Conv((3, 3, 1, 32), tf.nn.relu),
    Conv((3, 3, 32, 32), tf.nn.relu),
    Pooling((1, 2, 2, 1)),
    Dropout(dropout_rate=0.25),
    Conv((3, 3, 32, 64), tf.nn.relu),
    Conv((3, 3, 64, 64), tf.nn.relu),
    Pooling((1, 2, 2, 1)),
    Dropout(dropout_rate=0.5),
    Flatten(),
    Dense(4*4*64, 200, tf.nn.relu),
    Dropout(dropout_rate=0.5),
    Dense(200, 200, tf.nn.relu),
    Dropout(dropout_rate=0.5),
    Dense(200, 10, tf.nn.softmax)
]

lmd = 1e-4 # l2正則化係数

y = fprops(layers, x)
params_all = get_params(layers)
l2_reg = compute_l2_reg(params_all)

cost = - tf.reduce_mean(tf.reduce_sum(t * tf_log(y), axis=1)) + lmd * l2_reg
train = tf.train.AdamOptimizer(0.001).minimize(cost)

### 学習 ###

n_epochs = 60
batch_size = 100
n_batches = x_train.shape[0]//batch_size


sess = tf.Session()
sess.run(tf.global_variables_initializer())


for epoch in range(n_epochs):
    x_train, t_train = shuffle(x_train, t_train, random_state=random_state)
    for batch in range(n_batches):
        start = batch * batch_size
        end = start + batch_size
        sess.run(train, feed_dict={x: x_train[start:end], t: t_train[start:end], is_training: True})
            
    y_pred, cost_valid = sess.run([y, cost], feed_dict={x: x_valid, t: t_valid, is_training: False})
        
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
        epoch,
        cost_valid,
        accuracy_score(t_valid.argmax(axis=1), y_pred.argmax(axis=1))
    ))
    
    if accuracy_score(t_valid.argmax(axis=1), y_pred.argmax(axis=1)) >= 0.932:
        break
    
y_pred = sess.run(y, feed_dict={x: x_test, is_training: False})
submission = pd.Series(y_pred.argmax(axis=1), name='label')
submission.to_csv('/content/drive/My Drive/lecture_master/2019_summer/deeplearning_matsuo/homework/chap06/submission_pred.csv', header=True, index_label='id')

EPOCH: 0, Valid Cost: 0.633, Valid Accuracy: 0.798
EPOCH: 1, Valid Cost: 0.538, Valid Accuracy: 0.841
EPOCH: 2, Valid Cost: 0.469, Valid Accuracy: 0.863
EPOCH: 3, Valid Cost: 0.422, Valid Accuracy: 0.883
EPOCH: 4, Valid Cost: 0.408, Valid Accuracy: 0.883
EPOCH: 5, Valid Cost: 0.387, Valid Accuracy: 0.890
EPOCH: 6, Valid Cost: 0.373, Valid Accuracy: 0.898
EPOCH: 7, Valid Cost: 0.360, Valid Accuracy: 0.904
EPOCH: 8, Valid Cost: 0.347, Valid Accuracy: 0.909
EPOCH: 9, Valid Cost: 0.343, Valid Accuracy: 0.909
EPOCH: 10, Valid Cost: 0.340, Valid Accuracy: 0.911
EPOCH: 11, Valid Cost: 0.349, Valid Accuracy: 0.907
EPOCH: 12, Valid Cost: 0.335, Valid Accuracy: 0.913
EPOCH: 13, Valid Cost: 0.335, Valid Accuracy: 0.914
EPOCH: 14, Valid Cost: 0.316, Valid Accuracy: 0.921
EPOCH: 15, Valid Cost: 0.317, Valid Accuracy: 0.918
EPOCH: 16, Valid Cost: 0.329, Valid Accuracy: 0.918
EPOCH: 17, Valid Cost: 0.315, Valid Accuracy: 0.922
EPOCH: 18, Valid Cost: 0.320, Valid Accuracy: 0.918
EPOCH: 19, Valid Cost: